## Milestone 4: Querying the Data

**`Milestone 4 of the Multi Centralisation Data Project is within SQL on pgAdmin4 with the queries made added into the notebook.`**

As per the statement above, the below will show nine different tasks that were set by the fictional company's boss in order to analyse the data that has now been centralised.

Each task has a question associated in the title, as well as a `projected output`, the `query` that is ran on pgAdmin4, as well as an image of the `output` given from the query.

### Contents

- [__`Task 1`__](#task-1-How-many-stores-does-the-business-have-and-in-which-countries)
- [__`Task 2`__](#task-2-which-locations-currently-have-the-most-storestask-1-how-many-stores-does-the-business-have-and-in-which-countries)
- [__`Task 3`__](#task-3-which-months-produce-the-average-highest-cost-of-sales-typically)
- [__`Task 4`__](#task-4-how-many-sales-are-coming-from-online)
- [__`Task 5`__](#task-5-what-percentage-of-sales-come-through-each-type-of-store)
- [__`Task 6`__](#task-6-which-month-in-each-year-produced-the-highest-cost-of-sales)
- [__`Task 7`__](#task-7-what-is-the-total-staff-headcount-for-each-country-the-company-sells-in)
- [__`Task 8`__](#task-8-which-german-store-type-is-selling-the-most)
- [__`Task 9`__](#task-9-how-quickly-is-the-company-making-sales)

#### **Task 1: How many stores does the business have and in which countries?**

Projected:

    +----------+-----------------+
    | country  | total_no_stores |
    +----------+-----------------+
    | GB       |             265 |
    | DE       |             141 |
    | US       |              34 |
    +----------+-----------------+

Query:

    SELECT country_code, COUNT(*)
    FROM dim_store_details
    GROUP BY country_code;

Output:

![Task1](milestone_4_images/Task1.png)

The stores in GB has 1 more due to the dataset having the `WEB` store stated as being inside GB.

#### **Task 2: Which locations currently have the most stores?**

Projected:

    +-------------------+-----------------+
    |     locality      | total_no_stores |
    +-------------------+-----------------+
    | Chapletown        |              14 |
    | Belper            |              13 |
    | Bushley           |              12 |
    | Exeter            |              11 |
    | High Wycombe      |              10 |
    | Arbroath          |              10 |
    | Rutherglen        |              10 |
    +-------------------+-----------------+

    SELECT locality, COUNT(*) AS total_no_stores
    FROM dim_store_details
    GROUP BY locality
    ORDER BY total_no_stores DESC
    LIMIT 7;

Output:

![Task2](milestone_4_images/Task2.png)

#### **Task 3: Which months produce the average highest cost of sales typically?**


Projected:

    +-------------+-------+  
    | total_sales | month |  
    +-------------+-------+  
    |   673295.68 |     8 |  
    |   668041.45 |     1 |  
    |   657335.84 |    10 |  
    |   650321.43 |     5 |  
    |   645741.70 |     7 |  
    |   645463.00 |     3 |  
    +-------------+-------+  

Query:

    SELECT EXTRACT(MONTH FROM sale_timestamp) AS "month", 
        ROUND(SUM(product_quantity * "product_price (GBP)")::numeric, 2) AS total_sales
    FROM dim_date_times AS dt
    JOIN orders_table AS o
    ON dt.date_uuid = o.date_uuid
    JOIN dim_products AS p
    ON o.product_code = p.product_code
    GROUP BY EXTRACT(MONTH FROM sale_timestamp)
    ORDER BY total_sales DESC
    LIMIT 6;

Output:

![Task3](milestone_4_images/Task3.png)

There is a significant difference between the projected & what the query ran is showing, this may be due to less rows being dropped within the dataset I have cleaned compared to the projected output.

#### **Task 4: How many sales are coming from online?**

Projected:

    +------------------+-------------------------+----------+
    | numbers_of_sales | product_quantity_count  | location |
    +------------------+-------------------------+----------+
    |            26957 |                  107739 | Web      |
    |            93166 |                  374047 | Offline  |
    +------------------+-------------------------+----------+

Query:

    SELECT COUNT(*) AS number_of_sales, SUM(product_quantity) AS product_quantity_count,
    CASE
        WHEN store_code = 'WEB-1388012W' THEN 'Online'
        ELSE 'Offline'
    END AS "location"
    FROM orders_table
    GROUP BY 
    CASE
        WHEN store_code = 'WEB-1388012W' THEN 'Online'
        ELSE 'Offline'
    END;

Output:

![Task4](milestone_4_images/Task4.png)

#### **Task 5: What percentage of sales come through each type of store?**

Projected:

    +-------------+-------------+---------------------+
    | store_type  | total_sales | percentage_total(%) |
    +-------------+-------------+---------------------+
    | Local       |  3440896.52 |               44.87 |
    | Web portal  |  1726547.05 |               22.44 |
    | Super Store |  1224293.65 |               15.63 |
    | Mall Kiosk  |   698791.61 |                8.96 |
    | Outlet      |   631804.81 |                8.10 |
    +-------------+-------------+---------------------+

Query:

    SELECT
        sd.store_type,
        ROUND(SUM("product_price (GBP)" * o.product_quantity)::numeric, 2) AS total_sales,
        ROUND((SUM("product_price (GBP)" * o.product_quantity)::numeric / SUM(SUM("product_price (GBP)" * o.product_quantity)::numeric) OVER ()) * 100, 2) AS "percentage_total (%)"
    FROM dim_store_details sd
    JOIN orders_table o ON sd.store_code = o.store_code
    JOIN dim_products p ON o.product_code = p.product_code
    GROUP BY sd.store_type
    ORDER BY total_sales DESC;

Output:

![Task5](milestone_4_images/Task5.png)

#### **Task 6: Which month in each year produced the highest cost of sales?**

Projected:

    +-------------+------+-------+
    | total_sales | year | month |
    +-------------+------+-------+
    |    27936.77 | 1994 |     3 |
    |    27356.14 | 2019 |     1 |
    |    27091.67 | 2009 |     8 |
    |    26679.98 | 1997 |    11 |
    |    26310.97 | 2018 |    12 |
    |    26277.72 | 2019 |     8 |
    |    26236.67 | 2017 |     9 |
    |    25798.12 | 2010 |     5 |
    |    25648.29 | 1996 |     8 |
    |    25614.54 | 2000 |     1 |
    +-------------+------+-------+

Query:

    SELECT 
        ROUND(SUM("product_price (GBP)" * o.product_quantity)::numeric, 2) AS total_sales,
        EXTRACT(YEAR FROM dt.sale_timestamp) AS "year", 
        EXTRACT(MONTH FROM dt.sale_timestamp) AS "month" 
    FROM dim_date_times dt
    JOIN orders_table o
    ON dt.date_uuid = o.date_uuid
    JOIN dim_products p
    ON o.product_code = p.product_code
    GROUP BY EXTRACT(YEAR FROM dt.sale_timestamp), EXTRACT(MONTH FROM dt.sale_timestamp)
    ORDER BY total_sales DESC
    LIMIT 10;

Output:

![Task6](milestone_4_images/Task6.png)

#### **Task 7: What is the total staff headcount for each country the company sells in?**

Projected:

    +---------------------+--------------+
    | total_staff_numbers | country_code |
    +---------------------+--------------+
    |               13307 | GB           |
    |                6123 | DE           |
    |                1384 | US           |
    +---------------------+--------------+

Query:

    SELECT SUM(staff_numbers) AS total_staff_numbers, country_code
    FROM dim_store_details
    GROUP BY country_code
    ORDER BY total_staff_numbers DESC;

Output:

![Task7](milestone_4_images/Task7.png)

#### **Task 8: Which German store type is selling the most?**

Projected:

    +--------------+-------------+--------------+
    | total_sales  | store_type  | country_code |
    +--------------+-------------+--------------+
    |   198373.57  | Outlet      | DE           |
    |   247634.20  | Mall Kiosk  | DE           |
    |   384625.03  | Super Store | DE           |
    |  1109909.59  | Local       | DE           |
    +--------------+-------------+--------------+

Query:

    SELECT ROUND(SUM("product_price (GBP)" * product_quantity)::numeric, 2) AS total_sales, sd.store_type, sd.country_code
    FROM dim_store_details sd
    JOIN orders_table o
    ON sd.store_code = o.store_code
    JOIN dim_products p
    ON o.product_code = p.product_code
    GROUP BY sd.store_type, sd.country_code
    HAVING country_code = 'DE'
    ORDER BY total_sales;

Output:

![Task8](milestone_4_images/Task8.png)

#### Task 9: How quickly is the company making sales?

Projected:

    +------+-------------------------------------------------------+
    | year |                           actual_time_taken           |
    +------+-------------------------------------------------------+
    | 2013 | "hours": 2, "minutes": 17, "seconds": 12, "millise... |
    | 1993 | "hours": 2, "minutes": 15, "seconds": 35, "millise... |
    | 2002 | "hours": 2, "minutes": 13, "seconds": 50, "millise... | 
    | 2022 | "hours": 2, "minutes": 13, "seconds": 6,  "millise... |
    | 2008 | "hours": 2, "minutes": 13, "seconds": 2,  "millise... |
    +------+-------------------------------------------------------+

Query:

    WITH TimeDifference AS (
    SELECT EXTRACT(YEAR FROM sale_timestamp) AS "Year", sale_timestamp, 
        LEAD(sale_timestamp) OVER (
        ORDER BY sale_timestamp) - sale_timestamp AS difference
    FROM orders_table o
    JOIN dim_date_times dt
    ON o.date_uuid = dt.date_uuid
    JOIN dim_store_details sd
    on o.store_code = sd.store_code
    ORDER BY sale_timestamp)

    SELECT "Year", 
        concat('hours: ' || EXTRACT(HOUR FROM AVG(difference)) || ', ', 
            ' minutes: ' || EXTRACT(MINUTE FROM AVG(difference)) || ', ',
            ' seconds: ' || EXTRACT(SECOND FROM AVG(difference))::INTEGER || ', ',
            ' milliseconds: ' || EXTRACT(MILLISECOND FROM AVG(difference))) AS actual_time_taken
    FROM TimeDifference
    GROUP BY "Year"
    ORDER BY EXTRACT(MINUTE FROM AVG(difference)) DESC
    LIMIT 5

Output:

![Task9](milestone_4_images/Task9.png)

The actual output differs slightly from the projected output due to a nuance in the data cleaning process. In 2022, 2008, and 2002, the sales occurred within the same minute, which may lead to slight variations in their order compared to the projected results.